In [1]:
Hi Arran,

I've copied the 20/21 traverse data here:

/Volumes/arc_04/FIELD_DATA/K8622021/

Looks like metadata is in KIS\ Science/KIS_science.shp

Looks like the winter over station did well. Have a look at the point measurements when you get a chance and see if everything you are expecting is there.

On his own initiative all the stations were visited with the UC system. I'm hoping this doesn't cause any issues.

Cheers - Huw


SyntaxError: invalid syntax (<ipython-input-1-4802354451a7>, line 1)

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import glob
import os
import sys
import time
import datetime as dt
import pandas as pd
import geopandas as gpd
import scipy as sp
import rasterio as rio
from shapely.geometry import Point, LineString, Polygon, MultiPoint, box
import fiona
from shapely.ops import nearest_points
import matplotlib as mpl

In [3]:
!ls /Volumes/arc_04/FIELD_DATA/K8622021/'KIS Science'/

'ApRES data files'
 ApRES_RESOURCES
 K862_2021_ShapeFiles
 K862A_2021_TraverseScience-compressed.pdf
'KIS2 New ApRES Trial Sub burst results'
'KIS2 WO GPS copy'
 KIS_science.cpg
 KIS_science.dbf
 KIS_science.prj
 KIS_science.qpj
 KIS_science.shp
 KIS_science.shx
'KSI1 Winterover ApRES Trial subburst.docx'


In [508]:
metadata = gpd.read_file("/Volumes/arc_04/FIELD_DATA/K8622021/KIS Science/KIS_science.shp")


In [509]:
metadata.query('(Data_type=="ApRES") & (Project=="KIS2")',inplace=True)

In [510]:
metadata.drop(columns=['id', 'Uni_Event', 'Data_type',
       'Flag1ht_cm', 'Flag2ht_cm',  'Project'], 
             inplace=True)

In [511]:
metadata.keys()

Index(['Date', 'AF_Gain', 'Burst', 'RF_Atten', 'Depth_m', 'Site_ID', 'Notes',
       'ApRes_Unit', 'File_Name', 'Y', 'X', 'geometry'],
      dtype='object')

In [512]:
metadata.rename(columns={"X":"lon","Y":"lat",'Burst':"Stacks","ApRes_Unit":"APRES_unit","Depth_m":"Approx_Depth"},inplace=True)

In [513]:
metadata['Epoch'] = 3
metadata['utc_time'] = np.nan

In [514]:
metadata['Site_ID'] = metadata.Site_ID.apply(lambda x: x.upper()).copy()

In [515]:
metadata.loc[61,['Site_ID']] ='CP07'

In [516]:
metadata.drop(columns=['geometry'],inplace=True)

In [517]:
metadata.head()

,Date,AF_Gain,Stacks,RF_Atten,Approx_Depth,Site_ID,Notes,APRES_unit,File_Name,lat,lon,Epoch,utc_time
35,2020-12-22,-4,64.0,20,770.0,CP34,F1=W. New site,"UC Antenna, UC unit SN-143",Survey_2020-12-22_035737.dat,-82.471847,-152.208053,3,NaN
36,2020-12-22,-4,64.0,21,665.0,CP33,"F1=W. Two sharp reflectors at 665m and 764m, s...","UC Antenna, UC unit SN-143",Survey_2020-12-22_042049.dat,-82.471548,-152.224309,3,NaN
37,2020-12-22,-4,64.0,20,566.0,CP32,"F1=W. Two sharp reflectors at 566m 765m, new site","UC Antenna, UC unit SN-143",Survey_2020-12-22_044149.dat,-82.471325,-152.238891,3,NaN
38,2020-12-22,-4,64.0,20,493.0,CP31,"F1=W. Two sharp reflectors at 493m, 758m , new...","UC Antenna, UC unit SN-143",Survey_2020-12-22_050111.dat,-82.471154,-152.257888,3,NaN
39,2020-12-22,-4,64.0,24,447.0,CP30,"F1=W, new site, sharp reflectors at 447m and 750m","UC Antenna, UC unit SN-143",Survey_2020-12-22_052753.dat,-82.471072,-152.278526,3,NaN


In [518]:
metadata.APRES_unit.unique()

array(['UC Antenna, UC unit SN-143', None], dtype=object)

In [519]:
metadata1920 = pd.read_csv("/Users/home/whitefar/DATA/ApRES/K862A1920ApRESSites_KIS2.csv")
metadata1920.keys()

Index(['SiteName', 'YYYYUTC', 'MTUTC', 'DDUTC', 'HHUTC', 'MNUTC', 'LATDD',
       'LATMM', 'LONDD', 'LONMM', 'FILE', 'Atten', 'Gain', 'Stacks', 'Epoch',
       'ApproxDepth', 'checked', 'comments'],
      dtype='object')

In [520]:
metadata1920['utc_datetime'] = pd.to_datetime({'year':metadata1920.YYYYUTC,
                           'month':metadata1920.MTUTC,
                            'day':metadata1920.DDUTC,
                               'hour':metadata1920.HHUTC,
                            'minute':metadata1920.MNUTC}).apply(lambda x: pd.Timestamp(x))

In [521]:
metadata1920['utc_time'] = metadata1920['utc_datetime'].apply(lambda x: x.isoformat())
metadata1920['Date'] =metadata1920['utc_datetime'].apply(lambda x: x.date())

In [522]:
metadata1920['lat'] = (metadata1920.LATDD - metadata1920.LATMM/60).round(decimals=6)
metadata1920['lon'] = (metadata1920.LONDD - metadata1920.LONMM/60).round(decimals=6)

In [523]:
metadata1920['APRES_unit'] = 'Huws_unit1'

In [524]:
metadata1920.rename(columns={"SiteName":"Site_ID","FILE":"File_Name","Gain":"AF_Gain","Atten":"RF_Atten","ApproxDepth":"Approx_Depth","comments":"Notes"},inplace=True)

In [525]:
metadata1920.drop(columns=['utc_datetime'],inplace=True)

In [526]:
metadata1920.keys()

Index(['Site_ID', 'YYYYUTC', 'MTUTC', 'DDUTC', 'HHUTC', 'MNUTC', 'LATDD',
       'LATMM', 'LONDD', 'LONMM', 'File_Name', 'RF_Atten', 'AF_Gain', 'Stacks',
       'Epoch', 'Approx_Depth', 'checked', 'Notes', 'utc_time', 'Date', 'lat',
       'lon', 'APRES_unit'],
      dtype='object')

In [527]:
metadata1920 = metadata1920[metadata.keys()].copy()

In [528]:
meta_out = pd.concat([metadata1920,metadata])

In [529]:
meta_out.reset_index(drop=True,inplace=True)

In [530]:
meta_out.keys()

Index(['Date', 'AF_Gain', 'Stacks', 'RF_Atten', 'Approx_Depth', 'Site_ID',
       'Notes', 'APRES_unit', 'File_Name', 'lat', 'lon', 'Epoch', 'utc_time'],
      dtype='object')

In [531]:
meta_out = meta_out[['Date', 'utc_time', 'lat', 'lon','Approx_Depth','File_Name','AF_Gain', 'Stacks', 'RF_Atten',  'Site_ID',
       'Notes', 'APRES_unit',   'Epoch']]


In [532]:
meta_out.to_csv('/Users/home/whitefar/DATA/ApRES/kis2_apres_metadata.csv')

In [533]:
meta_out.Site_ID.unique()

array(['CP01', 'CP02', 'CP03', 'CP04', 'CP05', 'CP06', 'CP07', 'CP08',
       'CP09', 'CP10', 'CP11', 'CP12', 'CP13', 'CP14', 'CP15', 'CP16',
       'CP17', 'CP18', 'CP19', 'CP20', 'CP21', 'CP22', 'CP23', 'CP24',
       'CP25', 'CP34', 'CP33', 'CP32', 'CP31', 'CP30', 'CP_29_DRILL',
       'CP28', 'CP27', 'CP26'], dtype=object)

In [534]:
meta_out.sort_values(by=['Site_ID'],inplace=True)

In [535]:
meta_out.reset_index(drop=True,inplace=True)

In [536]:
meta_out.to_csv('/Users/home/whitefar/DATA/ApRES/kis2_apres_metadata_by_Site_ID.csv')

In [475]:
!ls /Volumes/arc_04/FIELD_DATA/K8622021/'KIS Science'/'ApRES data files'/KIS2

'KIS2 Old WO ApRES Card1'       Survey_2020-12-23_002658.dat
'KIS2 Old WO ApRES Card2'       Survey_2020-12-23_004352.dat
 Survey_2020-12-22_035737.dat   Survey_2020-12-23_010317.dat
 Survey_2020-12-22_042049.dat   Survey_2020-12-23_013605.dat
 Survey_2020-12-22_044149.dat   Survey_2020-12-23_015608.dat
 Survey_2020-12-22_050111.dat   Survey_2020-12-23_021900.dat
 Survey_2020-12-22_052753.dat   Survey_2020-12-23_023455.dat
 Survey_2020-12-22_201557.dat   Survey_2020-12-23_024953.dat
 Survey_2020-12-22_203746.dat   Survey_2020-12-23_030547.dat
 Survey_2020-12-22_205751.dat   Survey_2020-12-23_042754.dat
 Survey_2020-12-22_211915.dat   Survey_2020-12-23_045043.dat
 Survey_2020-12-22_213835.dat   Survey_2020-12-23_050820.dat
 Survey_2020-12-22_221342.dat   Survey_2020-12-23_052812.dat
 Survey_2020-12-22_223029.dat   Survey_2020-12-23_202213.dat
 Survey_2020-12-22_224858.dat   Survey_2020-12-23_204021.dat
 Survey_2020-12-22_230405.dat   Survey_2020-12-23_205701.dat
 Survey_2020-12-22_23324

In [507]:
meta_out.keys()

Index(['Date', 'utc_time', 'lat', 'lon', 'Approx_Depth', 'File_Name',
       'AF_Gain', 'Stacks', 'RF_Atten', 'Site_ID', 'Notes', 'APRES_unit',
       'Epoch'],
      dtype='object')